In [1]:
from flask import Flask, request,render_template, session
import psycopg2
import hashlib
import os

In [2]:
app = Flask(__name__)
app.config["CACHE_TYPE"] = "null"

In [3]:
# salt = os.urandom(32) # Remember this
salt = b'|\xb0\x84\x9d\xc8;\xd9}\x1a\xaf\xee\\H\xae\x17\xbb\\i\xb3\r\x11W\xf9\r1\x01\xabX}\xd4\x8b\xf7' 

def connect(database="database",user="cygne",password="0000", host="localhost",port="5433"):
    try: 
        conn = psycopg2.connect(database=database, user=user, password=password, host=host,port=port)
        print("connected")
    except:
        print ("I am unable to connect to the database")

    mycursor = conn.cursor()

    return conn,mycursor

def hashPassword(password):
   
    passwordHashed = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000)
    return passwordHashed.hex()

In [4]:
@app.route('/')
def index():
    return render_template('formulair2.html', msg="")

@app.route('/AjouterCompte', methods=['GET'])
def ajouterCompte():
    return render_template('ajout-compte.html')


@app.route('/Connect',methods=['GET', 'POST'])
def login():
    msg = ''
    if request.method == 'POST' and 'identifiant' in request.form and 'password' in request.form: 
        identifiant = request.form['identifiant']
        password = hashPassword(request.form['password'])
        conn,mycursor = connect()
        mycursor.execute('SELECT * FROM accounts WHERE identifiant = %s AND password = %s', (identifiant, password)) 
        account = mycursor.fetchone()
        
        if account:
            
            return "OK, connection reussie avec le username " + identifiant
        else:
            
            msg = 'Erreur. idantifiant ou mot de passe incorrect, recommencer !'
    
    return render_template('formulair2.html', msg=msg)
    

@app.route('/addRegion', methods=['POST'])
def ajouterCompteToDatabase():
    identifiant = request.form['identifiant']
    email = request.form['email']
    password = hashPassword(request.form['password'])
    
    conn,mycursor = connect() 
    requete = "INSERT INTO accounts (identifiant, email, password) VALUES (%s,%s,%s)"
    
    conn,mycursor = connect()
    mycursor.execute(requete,(identifiant,email,password))
    conn.commit()
    return render_template('formulair2.html')

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0',port=50000,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:50000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2021 14:16:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 14:17:23] "POST /Connect HTTP/1.1" 200 -


connected


127.0.0.1 - - [28/Oct/2021 14:17:27] "GET /AjouterCompte HTTP/1.1" 200 -
